# Quantum Teleportation Protocol – Final Project

**Topic:** Quantum Teleportation using Qiskit  
**Author:** Harish Kumar  
**Roll No:** M25IQT006  

This notebook implements the standard **quantum teleportation protocol**.

We will:

- Prepare an arbitrary single-qubit state $|\psi\rangle$
- Create an entangled Bell pair between Alice and Bob
- Teleport $|\psi\rangle$ from Alice’s qubit to Bob’s qubit
- Verify that Bob’s final state matches the original state using **state fidelity**

The notebook is structured as follows:

1. Install and import required libraries  
2. Define a helper function to prepare an arbitrary qubit state  
3. Build the teleportation circuit with **measurements and classical control**  
4. Build a **unitary-only** teleportation circuit for analysis  
5. Verify teleportation using **statevectors and fidelity**  
6. Run the simulation and interpret results  


In [1]:
# Install Qiskit and Qiskit-Aer (only needed in environments like Colab)
# In the course uv environment, these may already be installed.

# !pip install qiskit qiskit-aer


## 1. Imports and Basic Setup

In this cell, we import all the Python packages and Qiskit modules we will use:

- `QuantumCircuit`, `QuantumRegister`, `ClassicalRegister`, `transpile` for building circuits  
- `Statevector`, `DensityMatrix`, `partial_trace`, `state_fidelity` for quantum state analysis  
- `AerSimulator` for running simulations  
- `numpy` for numerical work  


In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import Statevector, state_fidelity, DensityMatrix, partial_trace
from qiskit_aer import AerSimulator
import numpy as np


## 2. Helper Function – Preparing an Arbitrary Single-Qubit State

Quantum teleportation works for **any** input qubit $|\psi\rangle$, not just $|0\rangle$ or $|1\rangle$.

We define a function:

$|\psi\rangle = \cos(\theta/2)|0\rangle + e^{i\phi}\sin(\theta/2)|1\rangle$

Using `RY(θ)` and `RZ(φ)` gates, we can place any pure state on a single qubit.

The function below:

- Takes a `QuantumCircuit`
- Applies rotations on a chosen qubit to prepare $|\psi\rangle$
- Parameters `theta` and `phi` control the state on the **Bloch sphere**


In [3]:
def prepare_unknown_state(qc: QuantumCircuit, qubit: int, theta: float, phi: float):
    """
    Prepare an arbitrary single-qubit state on 'qubit' of circuit qc:

        |ψ> = cos(θ/2)|0> + e^{iφ} sin(θ/2)|1>

    Parameters
    ----------
    qc : QuantumCircuit
        Circuit in which to prepare the state.
    qubit : int
        Index of the target qubit.
    theta : float
        Polar angle in [0, π].
    phi : float
        Azimuthal angle in [0, 2π).
    """
    qc.ry(theta, qubit)
    qc.rz(phi, qubit)



## 3. Teleportation Circuit with Measurements (Physical Protocol)

We now build the **actual teleportation protocol** with:

- 3 qubits:  
  - `q0`: Alice's unknown qubit \(|\psi\rangle\)  
  - `q1`: Alice's half of the entangled pair  
  - `q2`: Bob's half of the entangled pair (destination qubit)  

- 3 classical bits:  
  - `c0`: Result of measuring `q0` (Alice)  
  - `c1`: Result of measuring `q1` (Alice)  
  - `c_b`: Result of measuring `q2` (Bob), used for checking

**Protocol steps in this circuit:**

1. Prepare \(|\psi\rangle\) on `q0`  
2. Create a Bell pair between `q1` and `q2` (entanglement)  
3. Alice performs a Bell measurement on `q0` and `q1`  
4. Alice measures both qubits and stores results in `c0`, `c1`  
5. Based on these classical bits, Bob applies correction gates:
   - If `c0 == 1`, apply **X** to `q2`  
   - If `c1 == 1`, apply **Z** to `q2`  
6. Bob measures `q2` to verify the teleported state

We use `if_test` blocks to implement **classical conditional gates** in Qiskit.


In [4]:
def build_teleportation_circuit(theta: float, phi: float) -> QuantumCircuit:
    """
    Build the full teleportation circuit, including measurements and
    classical-conditional corrections on Bob's qubit.

    Qubit mapping
    -------------
    q0 : Alice's unknown qubit |ψ>
    q1 : Alice's half of the entangled pair
    q2 : Bob's half of the entangled pair (destination qubit)

    Classical registers
    -------------------
    c0 : measurement of q0 (Alice)
    c1 : measurement of q1 (Alice)
    c_b: measurement of q2 (Bob, for demonstration)
    """

    # Quantum and classical registers
    qr = QuantumRegister(3, "q")
    c0 = ClassicalRegister(1, "c0")   # Alice's result for qubit 0
    c1 = ClassicalRegister(1, "c1")   # Alice's result for qubit 1
    c_b = ClassicalRegister(1, "c_b") # Bob's final measurement

    qc = QuantumCircuit(qr, c0, c1, c_b, name="teleportation")

    # --- Step 1: Prepare unknown state |ψ> on qubit 0 ---
    prepare_unknown_state(qc, 0, theta, phi)

    # --- Step 2: Create entanglement between qubit 1 and 2 (Bell pair) ---
    qc.h(1)
    qc.cx(1, 2)

    # --- Step 3: Alice performs Bell-state measurement on qubits 0 and 1 ---
    qc.cx(0, 1)
    qc.h(0)

    # --- Step 4: Alice measures her qubits and stores results in c0, c1 ---
    qc.measure(0, c0[0])
    qc.measure(1, c1[0])

    # --- Step 5: Bob applies corrections based on Alice's classical bits ---
    # If c0 == 1, apply X to Bob's qubit
    with qc.if_test((c0, 1)):
        qc.x(qr[2])

    # If c1 == 1, apply Z to Bob's qubit
    with qc.if_test((c1, 1)):
        qc.z(qr[2])

    # --- Step 6: Bob measures his qubit (for demonstration) ---
    qc.measure(2, c_b[0])

    return qc


## 4. Unitary-Only Teleportation Circuit (for Analysis)

The previous circuit uses **measurements and classical control**, which is physically realistic.

For analysis, it is often useful to create a **purely unitary** circuit:

- No classical bits  
- No mid-circuit measurements  

Instead, we:

1. Prepare \(|\psi\rangle\) on qubit 0  
2. Create a Bell pair between qubits 1 and 2  
3. Apply the same teleportation operations  
4. Replace classical corrections with **coherent, controlled gates** (`CX` and `CZ`)  

This allows us to obtain a single **global statevector** and then trace out (ignore) Alice's qubits to study Bob’s final state.


In [5]:
def build_teleportation_unitary(theta: float, phi: float) -> QuantumCircuit:
    """
    Build a unitary-only version of teleportation (no classical bits).
    This is not the physical protocol but is mathematically equivalent
    and useful for verifying the final state of Bob's qubit.
    """

    qc = QuantumCircuit(3, name="teleportation_unitary")

    # Prepare unknown state on qubit 0
    prepare_unknown_state(qc, 0, theta, phi)

    # Create Bell pair on qubits 1 and 2
    qc.h(1)
    qc.cx(1, 2)

    # Teleportation steps
    qc.cx(0, 1)
    qc.h(0)

    # "Coherent" corrections:
    # Equivalent to the classical corrections after measurement
    qc.cx(1, 2)
    qc.cz(0, 2)

    return qc


## 5. Verifying Teleportation Using State Fidelity

To check if teleportation worked, we compare:

- **Target state**: $|\psi\rangle$ on a single qubit  
- **Bob's final state**: obtained from the 3-qubit teleportation circuit  

Steps:

1. Build a 1-qubit circuit with $|\psi\rangle$ and convert it to a `Statevector`  
2. Build the 3-qubit **unitary** teleportation circuit and get its `Statevector`  
3. Convert the target state to a `DensityMatrix`  
4. Use `partial_trace` to **trace out** Alice's qubits (0 and 1), leaving only Bob's qubit (2)  
5. Compute **state fidelity** between:
   - Bob's reduced state  
   - The target state $|\psi\rangle$  

A fidelity **close to 1** means teleportation was successful.


In [6]:
def verify_teleportation(theta: float, phi: float):
    """
    Verify teleportation by comparing Bob's final state to the original |ψ>.

    Prints:
    - Fidelity value (should be close to 1 for ideal teleportation).
    - Bob's reduced density matrix for reference.
    """

    # Target state |ψ> (1 qubit)
    qc_target = QuantumCircuit(1)
    prepare_unknown_state(qc_target, 0, theta, phi)
    psi_target = Statevector.from_instruction(qc_target)
    rho_target = DensityMatrix(psi_target)  # Convert to DensityMatrix explicitly

    # Teleportation unitary on 3 qubits
    qc_tele = build_teleportation_unitary(theta, phi)
    psi_tele_3 = Statevector.from_instruction(qc_tele)

    # Reduce to Bob's qubit (qubit 2) by tracing out qubits 0 and 1
    rho_bob = partial_trace(psi_tele_3, [0, 1])

    # Compute fidelity between Bob's state and target state
    fid = state_fidelity(rho_bob, rho_target)

    print("=== Teleportation Fidelity Check ===")
    print(f"theta = {theta:.4f}, phi = {phi:.4f}")
    print("Fidelity between original |ψ> and Bob's final state:", fid)
    print("\nBob's reduced density matrix:")
    print(rho_bob.data)
    print("====================================\n")


## 6. Running the Teleportation Simulation

Now we connect everything:

- Build the **measurement-based** teleportation circuit  
- Run it on `AerSimulator` with multiple shots  
- Print the measurement counts to see the distribution of:
  - Alice's classical bits
  - Bob's final measurement  

This helps us understand how different measurement outcomes occur and how the correction gates ensure Bob always recovers the state \(|\psi\rangle\) (up to global phase).


In [7]:
def run_simulation(theta: float, phi: float, shots: int = 2048):
    """
    Build the measurement-based teleportation circuit and run it on
    an ideal AerSimulator backend, printing the measurement statistics.
    """

    qc = build_teleportation_circuit(theta, phi)
    print("=== Teleportation Circuit (with measurements) ===")
    print(qc)
    print("================================================\n")

    simulator = AerSimulator()
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled, shots=shots).result()
    counts = result.get_counts()

    print(f"=== Measurement Counts (shots = {shots}) ===")
    print("Format of key: c_b c1 c0  (classical bits)")
    print(counts)
    print("============================================\n")


## 7. Main Function – Tie Everything Together

The `main()` function:

1. Chooses specific values of `theta` and `phi` to define the input state \(|\psi\rangle\)  
2. Runs the teleportation simulation with measurements  
3. Verifies teleportation using state fidelity in the unitary picture  

You can later:

- Change `theta` and `phi`  
- Call `run_simulation()` and `verify_teleportation()` with different states  


In [8]:
def main():
    """
    Main function for running the teleportation demo:
    - Picks an example |ψ> using theta, phi.
    - Runs the measurement-based teleportation.
    - Verifies the final state using fidelity.
    """

    # Choose parameters for the unknown state |ψ>
    # You can change these or randomize them for experiments.
    theta = np.pi / 3   # 60 degrees
    phi = np.pi / 5     # 36 degrees

    # 1) Run the teleportation protocol with measurements
    run_simulation(theta, phi, shots=2048)

    # 2) Verify using statevector fidelity
    verify_teleportation(theta, phi)


## 8. Run the Quantum Teleportation Demo

Now we call `main()` to:

- Build and display the teleportation circuit  
- Run the simulation on the quantum simulator  
- Compute and print fidelity between the original state and Bob's final state  

**If the code and environment are correct, the fidelity should be very close to 1.**


In [9]:
main()


=== Teleportation Circuit (with measurements) ===
       ┌─────────┐┌─────────┐     ┌───┐┌─┐                                    »
  q_0: ┤ Ry(π/3) ├┤ Rz(π/5) ├──■──┤ H ├┤M├────────────────────────────────────»
       └──┬───┬──┘└─────────┘┌─┴─┐└┬─┬┘└╥┘                                    »
  q_1: ───┤ H ├────────■─────┤ X ├─┤M├──╫─────────────────────────────────────»
          └───┘      ┌─┴─┐   └───┘ └╥┘  ║ ┌────── ┌───┐ ───────┐ ┌────── ┌───┐»
  q_2: ──────────────┤ X ├──────────╫───╫─┤ If-0  ┤ X ├  End-0 ├─┤ If-0  ┤ Z ├»
                     └───┘          ║   ║ └──╥─── └───┘ ───────┘ └──╥─── └───┘»
                                    ║   ║ ┌──╨──┐                   ║         »
 c0: 1/═════════════════════════════╬═══╩═╡ 0x1 ╞═══════════════════╬═════════»
                                    ║   0 └─────┘                ┌──╨──┐      »
 c1: 1/═════════════════════════════╩════════════════════════════╡ 0x1 ╞══════»
                                    0                            └────